In [1]:
from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs
import os, subprocess
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
subprocess.run(
        [
            "fluidsynth",
            "-q",
            "-ni",
            "models/FluidR3_GM.sf2",
            "whatever.mid",
            "-F",
            "whatever.wav",
            "-r",
            "16000",
        ],
        stderr=subprocess.DEVNULL,
    )
loader = MonoLoader(sampleRate=16000, resampleQuality=1)
loader.configure(filename="whatever.wav")
audio = loader()
del loader

2025-02-15 11:58:24.347288: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2025-02-15 11:58:24.707471: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-15 11:58:24.710647: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2025-02-15 11:58:24.973174: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-15 11:58:24.973199: I tensorflow/co

In [2]:
embedding_model = TensorflowPredictEffnetDiscogs(
    graphFilename="models/discogs-effnet-bs64-1.pb", output="PartitionedCall:1"
)
embeddings = embedding_model(audio)
del embedding_model

2025-02-15 11:58:28.437270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-15 11:58:28.437313: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 4090 computeCapability: 8.9
coreClock: 2.52GHz coreCount: 128 deviceMemorySize: 23.99GiB deviceMemoryBandwidth: 938.86GiB/s
2025-02-15 11:58:28.437328: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-15 11:58:28.437335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-15 11:58:28.437337: I tenso

In [7]:
from essentia.standard import TensorflowPredict2D
import json, numpy as np
mood = TensorflowPredict2D(graphFilename="models/mtg_jamendo_moodtheme-discogs-effnet-1.pb")
genre = TensorflowPredict2D(graphFilename="models/mtg_jamendo_genre-discogs-effnet-1.pb")

def get_mtg_tags(embeddings, tag_model, tag_json, max_num_tags=5, tag_threshold=0.01):
    # TODO stop opening me every run
    with open(tag_json, "r") as json_file:
        metadata = json.load(json_file)
    predictions = tag_model(embeddings)
    mean_act = np.mean(predictions, 0)
    ind = np.argpartition(mean_act, -max_num_tags)[-max_num_tags:]
    tags = []
    confidence_score = []
    for i in ind:
        # print(metadata['classes'][i] + str(mean_act[i]))
        if mean_act[i] > tag_threshold:
            tags.append(metadata["classes"][i])
            confidence_score.append(mean_act[i])
    ind = np.argsort(-np.array(confidence_score))
    tags = [tags[i] for i in ind]
    confidence_score = np.round((np.array(confidence_score)[ind]).tolist(), 4).tolist()

    return tags, confidence_score

mood_tags = get_mtg_tags(
    embeddings, mood, "models/mtg_jamendo_moodtheme-discogs-effnet-1.json"
)
genre_tags = get_mtg_tags(
    embeddings, genre, "models/mtg_jamendo_genre-discogs-effnet-1.json"
)
print(mood_tags)
print(genre_tags)

(['dark', 'film', 'meditative', 'relaxing', 'soundscape'], [0.1175, 0.1124, 0.0789, 0.0785, 0.065])
(['experimental', 'ambient', 'electronic', 'alternative', 'classical'], [0.4462, 0.2641, 0.257, 0.2537, 0.1244])


2025-02-15 11:58:35.904006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-15 11:58:35.904043: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 4090 computeCapability: 8.9
coreClock: 2.52GHz coreCount: 128 deviceMemorySize: 23.99GiB deviceMemoryBandwidth: 938.86GiB/s
2025-02-15 11:58:35.904056: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-15 11:58:35.904063: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-15 11:58:35.904065: I tenso

: 